# Tadmixture demo for the 1000 Genomes project

Simple script for the exemplary application of Tadmixture to the 1000 Genomes 
project, phase 3. The SNPs are reduced to SNPs in Kidd's AIMs panel, AMR populations 
are excluded resulting in 2157 sampled individuals.

    1. Set parameters
    2. Load data
    3. Preprocess data
    4. Constructing cuts
    5. Computing costs
    6. Constructing the tangles tree: For each cut compute the tangles by expanding on 
        the previous ones if it is consistent. If its not possible stop
    7. Read out typical genome 
    8. Compute characterizing cuts
    9. Postprocess in soft clustering and plot


In [48]:
import pickle
import sys
from functools import partial
import numpy as np
import pandas as pd
import compute_kNN
import plot_soft_clustering
import read_vcf
import reliability_factor
from src import cost_functions, data_types
from src import utils
from src.tree_tangles import (ContractedTangleTree, tangle_computation, 
                              compute_soft_predictions_children_popgen)
from src.utils import merge_doubles
sys.path.append('..')

### Set parameters.

In [49]:
n = 2157                    # number of individuals
agreement = 225             # agreement parameter
cost_fct = "FST_kNN"   # choice of cost function
k = 40                      # k for k-nearest neighbours
pruning = 0                 # pruning parameter

### Load data.

In [50]:
# load data from 1000 Genomes project (phase 3) reduced to SNPs in Kidd's AIMs panel,
# AMR populations excluded:
aims_data = read_vcf.ReadVCF(n, 'AIMs', 'AIMs_kidd_no_AMR', 'data/vcf/')
aims_data.read_vcf()

# get genotype matrix and embed data into tangles framework:
xs = np.transpose(aims_data.G[0])   # diploid genotype matrix
data = data_types.Data(xs=xs)       # embed aims_data in tangles framework

### Preprocess data.
First, compute a set of bipartitions or cuts from the data:

In [51]:
print("Generating set of bipartitions.", flush=True)
bipartitions = data_types.Cuts(values=(data.xs > 0).T,
                                names=np.array(list(range(0, data.xs.shape[1]))))
print("Found {} bipartitions.".format(len(bipartitions.values)), flush=True)

Generating set of bipartitions.
Found 55 bipartitions.


Secondly, since the individuals in aims_data are sorted according to their 
population membership, the vector of population memberships must be 
sorted in the same way:

In [52]:
# get population membership for each individual (individuals in vcf file are also
# sorted by populations, i.e. same sorting):
panel_file = ('data/vcf/integrated_call_samples_v3.20130502.ALL.panel')
panel_df = pd.read_csv(panel_file, delimiter='\t')
sample_pop = panel_df['pop']
pop_array = np.array(sample_pop)
custom_order = {'YRI': 0, 'LWK': 1, 'GWD': 2, 'MSL': 3, 'ESN': 4, 'ASW': 5,
                'ACB': 6, 'FIN': 7, 'CEU': 8, 'GBR': 9, 'TSI': 10, 'IBS': 11,
                'GIH': 12, 'PJL': 13, 'BEB': 14, 'STU': 15, 'ITU': 16, 'CHB': 17,
                'JPT': 18, 'CHS': 19, 'CDX': 20, 'KHV': 21, 'MXL': 22, 'PUR': 23,
                'CLM': 24, 'PEL': 25}
custom_sort_criteria = np.array([custom_order[pop] for pop in pop_array])
sorted_indices = np.argsort(custom_sort_criteria)
pop_membership = pop_array[sorted_indices]
pop_membership = pop_membership[:2157]  # exlcude AMR pops from pop_membership array
print("population membership of individuals after resorting (AMR populations excluded)"
      ":", pop_membership, flush=True)

population membership of individuals after resorting (AMR populations excluded): ['YRI' 'YRI' 'YRI' ... 'KHV' 'KHV' 'KHV']


### Compute costs.
The cost function uses the k-nearest neighbours matrix, which has to be pre-computed:

In [53]:
kNN = compute_kNN.KNearestNeighbours(xs, k, filename="kNN_AIMs_panel",
                                             filepath="data/saved_kNN/")
kNN.compute_kNN()
# pickle kNN-Matrix for cost function to load:
with open("data/saved_kNN/kNN", 'wb') as outp:
    pickle.dump(kNN, outp, pickle.HIGHEST_PROTOCOL)

With this, we can compute the costs of all cuts and sort them accordingly:

In [54]:
print("Calculating costs of bipartitions.")
cost_function = getattr(cost_functions, cost_fct)
bipartitions = utils.compute_cost_and_order_cuts(bipartitions, partial(cost_function,
                                                                       data.xs, None))

Calculating costs of bipartitions.
Computing costs of cuts...


100%|██████████| 55/55 [00:01<00:00, 30.70it/s]


Check for duplicate cuts and merge if necessary (this step can reduce the run time 
significantly:

In [55]:
print("Merging doublicate mutations.")
bipartitions = merge_doubles(bipartitions)
print("Number of bipartitions after merging:", len(bipartitions.names))

Merging doublicate mutations.
Number of bipartitions after merging: 55


### Construct tangles search tree.

In [ ]:
print("Building the tangle search tree", flush=True)
tangles_tree = tangle_computation(cuts=bipartitions, agreement=agreement, verbose=0)
print("Built tree has {} leaves".format(len(tangles_tree.maximals)), flush=True)
### uncomment if you have graphvis installed on your system
tangles_tree.plot_tree()

Building the tangle search tree
Built tree has 4 leaves


Postprocess the tangles search tree by contracting paths:

In [ ]:
# postprocess tree
print("Postprocessing the tree.", flush=True)
# contract to binary tree
print("Contracting to binary tree", flush=True)
contracted_tree = ContractedTangleTree(tangles_tree)
### uncomment if you have graphvis installed on your system
contracted_tree.plot_tree()

### Read out typical genome per population. 

In [ ]:
# identified populations (assign automatically in the future): 
id_pops = ["EAS", "SAS", "EUR", "AFR"]
# read out typical genome per population:
for c in range(len(tangles_tree.maximals)):
    print("typical genome of population", id_pops[c], ":",tangles_tree.maximals[c]
          .tangle.get_specification(), flush=True)
print("0 means that the typical genome is homozygous for the ancestral"
      " allele at the SNP corresponding to the SNP index, otherwise 1.", flush=True)
# print corresponding SNP index:
print("\nSNP index in typical genomes:\n", tangles_tree.maximals[0].tangle.get_cuts
().names, flush=True)

### Calculate set of characterizing cuts.

In [ ]:
# calculate set of characterizing cuts:
print("Calculating set of characterizing bipartitions.", flush=True)
contracted_tree.calculate_setP()

In [ ]:
# prune short paths:
contracted_tree.prune(bipartitions, pruning)
contracted_tree.calculate_setP()

## Compute soft clustering.

In [ ]:
# assign weight/ importance to bipartitions. no weights used for soft clustering in population genetics:
weight = np.ones(len(bipartitions.names))
# calculate the soft clustering, i.e. soft predictions:
print("Calculating soft predictions", flush=True)
compute_soft_predictions_children_popgen(node=contracted_tree.root, 
                                         cuts=bipartitions, weight=weight, 
                                         cuts_probs=reliability_factor.compute_reliability(
                                                 xs), verbose=3)
contracted_tree.processed_soft_prediction = True

Convert the soft clustering to be able to plot in a stacked bar plot:

In [ ]:
# get matrix with hierarchical soft clustering along the tangles tree to plot
# this inferred ancestry. char_cuts are the characteristic SNPs per split in
# the tangles tree, posititions indicate the position of the split in the tree.
matrices, char_cuts, positions = contracted_tree.to_matrix()

# get number of characterizing SNPs per split (necessary as bipartitions have been 
# merged):
num_char_cuts_per_split = []
for k in range(1, len(list(char_cuts.keys())) + 1):
    num_char_cuts_per_split.append(np.sum(np.array( [name.count(",") + 1 for name in 
                                                     bipartitions.names[list(char_cuts[k].keys())]])))
num_char_cuts = dict(zip(char_cuts.keys(), num_char_cuts_per_split))
print("number of characterizing cuts per split in tangles tree:", 
      num_char_cuts_per_split, flush=True)

Calculate hard predictions:

In [ ]:
print("Calculating hard predictions", flush=True)
ys_predicted, _ = utils.compute_hard_predictions(contracted_tree, cuts=bipartitions)

## Plot the soft clustering.

In [ ]:
print("Plotting the soft clustering, this step may take about 30 seconds.", flush=True)
plot_soft_clustering.plot_inferred_ancestry(matrices, pop_membership, agreement, "readVCF",
                                            cost_fct=cost_fct)